In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [3]:
# Step 1: Fetch the main program page
url = "https://2024.pycon.org.au/program/list/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 2: Extract URLs within the <main> section
main_section = soup.find('main')
session_links = main_section.find_all('a', href=True)

# Step 3 & 4: Extract the title and div content from each session page
data = []
base_url = "https://2024.pycon.org.au"

for link in session_links:
    session_url = base_url + link['href']
    session_response = requests.get(session_url)
    session_soup = BeautifulSoup(session_response.text, 'html.parser')
    
    # Extract title
    title = session_soup.find('title').text.strip()
    
    # Extract content in the div with class "lede astro-QLKTC6HR"
    lede_content = session_soup.find('div', class_='lede astro-QLKTC6HR')
    if lede_content:
        description = lede_content.text.strip()
    else:
        description = "No description available"

    # Extract name in the div with class "astro-B3UUC2C6"
    name = session_soup.find('b', class_='astro-B3UUC2C6')
    if name:
        name = name.text.strip()
    else:
        name = "No name available"
    
    data.append([title, name, description])

# Step 5: Create DataFrame
df = pd.DataFrame(data, columns=['Title','Name','Description'])


In [4]:
df.to_csv("pycon_2024_sessions.csv", index=False)

In [8]:
# import streamlit as st
# import pandas as pd
# from sentence_transformers import SentenceTransformer, util
# import torch

# # Load your data
# # df = pd.read_csv('session_data.csv')  # Assuming you have the DataFrame from the previous step

# # Load a pre-trained sentence transformer model
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # Compute embeddings for all descriptions
# embeddings = model.encode(df['Description'].tolist(), convert_to_tensor=True)

# # Streamlit UI to select a title
# selected_title = st.selectbox("Select a session title", df['Title'].values)

# # Get the selected description
# selected_description = df[df['Title'] == selected_title]['Description'].values[0]

# # Compute embedding for the selected description
# selected_embedding = model.encode(selected_description, convert_to_tensor=True)

# # Compute cosine similarities between the selected description and all others
# cosine_similarities = util.pytorch_cos_sim(selected_embedding, embeddings).flatten()

# # Find the top N most similar descriptions
# N = st.slider("Number of similar sessions to display", 1, 10, 5)
# top_n_indices = torch.topk(cosine_similarities, N + 1).indices[1:]  # Skip the first as it's the selected one

# # Display the most similar descriptions
# st.write(f"Top {N} most similar sessions to '{selected_title}':")
# for index in top_n_indices:
#     index = int(index)  # Convert tensor index to int
#     st.write(f"**{df.iloc[index]['Title']}**")
#     st.write(df.iloc[index]['Description'])
#     st.write("----")
